<a href="https://colab.research.google.com/github/Sanan-Qureshi/Color_Prediction_TensorFlow/blob/main/ML_Color_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we are going to write a code to upload our ZIP file.

In [ ]:
from google.colab import files
import zipfile
import os

# Upload the ZIP file
uploaded = files.upload()  # Select color.zip

# Unzip the folder
with zipfile.ZipFile('color.zip', 'r') as zip_ref:
    zip_ref.extractall('color')

# Check contents
os.listdir('color')


Saving color.zip to color.zip


['color']

Then let us check the uploaded file.

In [ ]:
import os

for root, dirs, files in os.walk('color'):
    print(root)


color
color/color
color/color/Red
color/color/Blue


If the output is like given below:
<br>color
<br>color/color
<br>color/color/Red
<br>color/color/Blue
<br> (If that's not the case, then please skip the two cell given below and move <br>onto TensorFlow part).
<br>Then, we need to run the code given below:


In [ ]:
import shutil
import os

# Move contents of color/color/ to color/
inner_folder = 'color/color'
for folder in os.listdir(inner_folder):
    shutil.move(os.path.join(inner_folder, folder), 'color')

# Now remove the empty nested folder
shutil.rmtree(inner_folder)


In [ ]:
for root, dirs, files in os.walk('color'):
    print(root)


color
color/Red
color/Blue


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 64, 64
batch_size = 8

train_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_gen.flow_from_directory(
    'color',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = train_gen.flow_from_directory(
    'color',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 48 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [ ]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, validation_data=val_data, epochs=5)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6938 - loss: 0.4616

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.7196 - loss: 0.4353 - val_accuracy: 1.0000 - val_loss: 0.0071
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 1.0000 - val_loss: 3.6868e-05
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 1.7277e-05 - val_accuracy: 1.0000 - val_loss: 6.9225e-07
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 1.0000 - loss: 3.9624e-07 - val_accuracy: 1.0000 - val_loss: 5.4167e-08
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 1.0000 - loss: 3.1677e-08 - val_accuracy: 1.0000 - val_loss: 1.1251e-08


In [ ]:
# Upload test image
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded = files.upload()  # Student uploads a test image

for filename in uploaded.keys():
    img_path = filename
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0  # Scale pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)

    if prediction[0][0] > 0.5:
        print("The image is most likely: 🔴 RED")
    else:
        print("The image is most likely: 🔵 BLUE")


Saving test.png to test.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
The image is most likely: 🔴 RED
